In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

### 1. ITEMID 선정

In [120]:
# Lab Events 가져오기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")

In [121]:
# result 1

# LABEVENTS에서 가장 빈도가 높은 상위 30개 ITEMID만 사용
# common_id = df['ITEMID'].value_counts().index[:30].to_list()

# LABEVENTS의 모든 ITEMID 사용
common_id = df['ITEMID'].value_counts().index.to_list()

df1 = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_LABITEMS.csv")
df2 = pd.merge(df1, pd.DataFrame(common_id, columns=["ITEMID"]), on="ITEMID")
result1 = df2[["ITEMID", "LABEL", "FLUID", "CATEGORY", "LOINC_CODE"]]
result1

,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN
...,...,...,...,...,...
721,51534,MYELOS,JOINT FLUID,HEMATOLOGY,NaN
722,51535,CD55,OTHER BODY FLUID,HEMATOLOGY,NaN
723,51536,CD59,OTHER BODY FLUID,HEMATOLOGY,NaN
724,51537,TDT,OTHER BODY FLUID,HEMATOLOGY,NaN


In [122]:
# result 2

# ARDS 참고 논문의 12개 Feature extract
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_LABITEMS.csv")
L_list = ["WBC Count", "WBC", "Creatinine, Serum", "Albumin", "Platelet Count", "pH", "Lactate", "Oxygen Saturation", "Oxygen", "pCO2", "heart rate", "Temperature"]
result2 = df[df['LABEL'].isin(L_list)]
result2

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
140,14,50813,Lactate,Blood,Blood Gas,32693-4
143,17,50816,Oxygen,Blood,Blood Gas,19994-3
144,18,50817,Oxygen Saturation,Blood,Blood Gas,20564-1
145,19,50818,pCO2,Blood,Blood Gas,11557-6
147,21,50820,pH,Blood,Blood Gas,11558-4
152,26,50825,Temperature,Blood,Blood Gas,NaN
158,32,50831,pH,Other Body Fluid,Blood Gas,2748-2
189,63,50862,Albumin,Blood,Chemistry,1751-7
407,281,51081,"Creatinine, Serum",Urine,Chemistry,NaN
420,294,51094,pH,Urine,Chemistry,2756-5


In [123]:
# result1&result2 결합
result = pd.concat([result1, result2], axis=0)
r_list = result["ITEMID"].tolist()
print("결합 후:",len(r_list))

# 겹치는 값 제거
result = result.drop_duplicates(subset=['ITEMID'])

# ITEMID 리스트에 담기
i_list = result["ITEMID"].tolist()
print("제거 후:",len(i_list))
result

결합 후: 740
제거 후: 726


,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE,ROW_ID
0,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3,NaN
1,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5,NaN
2,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2,NaN
3,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6,NaN
4,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN,NaN
...,...,...,...,...,...,...
721,51534,MYELOS,JOINT FLUID,HEMATOLOGY,NaN,NaN
722,51535,CD55,OTHER BODY FLUID,HEMATOLOGY,NaN,NaN
723,51536,CD59,OTHER BODY FLUID,HEMATOLOGY,NaN,NaN
724,51537,TDT,OTHER BODY FLUID,HEMATOLOGY,NaN,NaN


### 2. Data 가져오기

In [124]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/LABEVENTS.csv")

In [125]:
# 선정된 ITEMID와 겹치는 데이터 추출
df1 = df[df['ITEMID'].isin(r_list)]

# patient table(호흡기 질환 진단 받은 환자)에서 겹치는 데이터만 남기기 위해 HADM_ID int로 변경
df1['HADM_ID'] = df1['HADM_ID'].fillna(0).astype(int)
df1

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,0,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,0,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,0,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,0,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,0,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


In [126]:
# ARDS 환자 ID와 겹치는 데이터만 남기기
patients = pd.read_csv("~/project/MIMIC-III/ARDS.csv")

# 조인하여 데이터 추출
merged = pd.merge(df1, patients, on=['SUBJECT_ID', 'HADM_ID'], how='inner')

# 필요한 칼럼만 선택 + SUBJECT_ID 기준 정렬
df3 = merged[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM']]
df3 = df3.sort_values(by='SUBJECT_ID', ascending=True)
df3

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM
0,91,121205,51275,2177-04-27 05:00:00,42.9,42.9
895,91,121205,51275,2177-05-08 22:00:00,40.0,40.0
894,91,121205,51274,2177-05-08 22:00:00,15.7,15.7
893,91,121205,51265,2177-05-08 22:00:00,59,59.0
892,91,121205,51250,2177-05-08 22:00:00,87,87.0
...,...,...,...,...,...,...
4784871,99985,176670,51246,2181-02-11 05:35:00,OCCASIONAL,NaN
4784870,99985,176670,51244,2181-02-11 05:35:00,9,9.0
4784869,99985,176670,51237,2181-02-11 05:35:00,1.3,1.3
4784875,99985,176670,51251,2181-02-11 05:35:00,0,0.0


In [ ]:
# 환자별 첫 24시간 이내의 데이터만 남기기
df3['CHARTTIME'] = pd.to_datetime(df3['CHARTTIME']) # CHARTTIME 컬럼을 datetime 자료형으로 변환

df4 = df3.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).apply(lambda x: x[(x['CHARTTIME'] == x['CHARTTIME'].min()) | ((x['CHARTTIME'] > x['CHARTTIME'].min()) & (x['CHARTTIME'] <= x['CHARTTIME'].min() + pd.Timedelta(hours=24)))])
df4 = df4.reset_index(drop=True)
df4

###  3. Mean, Min, Max + Missing Value

In [ ]:
################ Mean, Min, Max 설정 ################

# 환자별 첫 24시간 이내의 데이터 구하기
df4.fillna(0, inplace=True)  # 결측치를 0으로 변경
df_mean = df4.groupby(['SUBJECT_ID', 'HADM_ID', 'ITEMID'], as_index=False).agg({'VALUENUM': lambda x: np.max(x.astype(float))})
df_mean

In [ ]:
# 최초 laboratory value의 pivot table 생성
pivot_table = df_mean.pivot_table(index=['SUBJECT_ID', 'HADM_ID'], columns='ITEMID', values='VALUENUM', aggfunc='first')
pivot_table = pivot_table.fillna(0)
df_pivot = pivot_table.reset_index(level=['SUBJECT_ID', 'HADM_ID'])
df_pivot

In [ ]:
# 환자별 laboratory value 추가하기
patients = patients.sort_values(by='SUBJECT_ID', ascending=True)
patients = pd.merge(patients[["SUBJECT_ID", "HADM_ID"]], df_pivot, on=['SUBJECT_ID', 'HADM_ID'], how='left')
patients

In [ ]:
# 칼럼 별로 관측되지 않은 laboratory value 값(0) 비율 확인 
zero_ratios = (patients == 0).sum() / len(patients)
zero_ratios = zero_ratios.to_frame()[2:]
zero_ratios = zero_ratios.reset_index().rename(columns={'index': 'ITEMID', 0:'ZERO_RATIOS'})
zero_ratios = pd.merge(zero_ratios, result[["ITEMID", "LABEL", "FLUID"]], on='ITEMID')
zero_ratios = zero_ratios.reindex(columns=["ITEMID", "LABEL", "FLUID", "ZERO_RATIOS"])
zero_ratios = zero_ratios.sort_values(by='ZERO_RATIOS', ascending=True)
zero_ratios

In [ ]:
################ Missing Value 설정 ################


# 관측되지 않은 값이 n% 미만인 ITEMID만 선정
data = zero_ratios[zero_ratios['ZERO_RATIOS'] <= 0.01]
below_list = data['ITEMID'].tolist()
print("ITEMID 개수:", len(below_list))
data

In [ ]:
# mean_patients에서 관측되지 않은 값이 n%이하인 칼럼의 값만 가져오기
i_list = ['SUBJECT_ID', 'HADM_ID']
i_list.extend(data['ITEMID'].tolist())

result = patients[i_list]

# 칼럼 명 변경을 위해 dict 생성 후 rename
id_dict = dict(zip(data['ITEMID'], data['LABEL']))
result = result.rename(columns=id_dict)
result

In [ ]:
result.to_csv("~/project/MIMIC-III/Data/FINAL/New_Lab/LABORATORY_VALUE_24h(1%)_max.csv", index=False)